In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import pandas as pd

# Kết nối với Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import ViTForImageClassification, ViTFeatureExtractor, Trainer, TrainingArguments
import torchvision.transforms as transforms
import datasets
import os
from datasets import load_dataset

In [ ]:
# Load dataset from folder
dataset = load_dataset("imagefolder", data_dir="/content/drive/MyDrive/splits")

# Define preprocessing
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
def transform(example):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std),
    ])
    example["pixel_values"] = [transform(img.convert("RGB")) for img in example["image"]]
    return example

dataset = dataset.map(transform, batched=True)
dataset = dataset.remove_columns(["image"]).rename_column("label", "labels")

Resolving data files:   0%|          | 0/240 [00:00<?, ?it/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model
num_labels = len(dataset["train"].features["labels"].names)
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k", num_labels=num_labels)
model.to(device)

training_args = TrainingArguments(
        output_dir="models/outputs",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=5,
        save_total_limit=2,
        logging_dir="logs",
        logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=feature_extractor,
)

trainer.train()
model.save_pretrained("models/outputs/vit_finetuned")

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-27-81c6c68214d5>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.105800,0.072441
2,0.026000,0.025808
3,0.017400,0.018910
4,0.015100,0.016590
5,0.014100,0.015894


In [ ]:
def evaluate():
    metrics = trainer.evaluate()
    print("Evaluation metrics:", metrics)
    return metrics

In [ ]:
def predict(image_path):
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std),
    ])
    pixel_values = transform(image).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(pixel_values)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=-1).item()

    label_names = dataset["train"].features["labels"].names
    return label_names[predicted_class]

In [ ]:
evaluate()

Evaluation metrics: {'eval_loss': 0.015894265845417976, 'eval_runtime': 7.957, 'eval_samples_per_second': 1.508, 'eval_steps_per_second': 0.251, 'epoch': 5.0}


{'eval_loss': 0.015894265845417976,
 'eval_runtime': 7.957,
 'eval_samples_per_second': 1.508,
 'eval_steps_per_second': 0.251,
 'epoch': 5.0}